# Features Analysis (step 2)

In [2]:
"""
NOTE: to run notebook use: venv/bin/python manage.py shell_plus --notebook
"""

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# plotting settings
sns.set(style="white", palette="muted", color_codes=True)
PLOTS_PATH = 'output/travel/'
PLOTS_FORMATS = ['.pdf', '.png']